---

### BA5E Find a Highest-Scoring Alignment of Two Strings
http://rosalind.info/problems/ba5e/

   
**Global Alignment Problem**
Find the highest-scoring alignment between two strings using a scoring matrix.

Given: Two amino acid strings.

Return: The maximum alignment score of these strings followed by an alignment achieving this maximum score. Use the BLOSUM62 scoring matrix and indel penalty σ = 5. (If multiple alignments achieving the maximum score exist, you may return any one.)

In [2]:
def scoreAlignment(w,v,gap=0,mismatch=0,match=1):
    """
    """
    ###
    wh = '-'+ w
    vh = '-'+ v
    n=len(vh)
    m=len(wh)
    ###
    scoring = pd.DataFrame(0,index=range(0,n),columns=range(0,m))
    ## Initialize F matrix
    for i in range(1,n):
        if vh[i] != wh[0]: scoring[0][i] = scoring[0][i-1] + gap
    for j in range(1,m):
        if vh[0] != wh[j]: scoring[j][0] = scoring[j-1][0] + gap

    ### Dynamic programming. Iterate over the scoring matrix.
    for i in range(1,n):
        for j in range(1,m):
            down = scoring[j][i-1] + gap
            right = scoring[j-1][i] + gap
            if vh[i] == wh[j]:
                diag = scoring[j-1][i-1] + match
            else:
                diag = scoring[j-1][i-1] + mismatch
            t=[('s',down),('e',right),('d',diag)]
            ### Get which one is max
            whichMax = np.argmax([_[1]for _ in t])                
            scoring[j][i] = t[whichMax][1]
    s2 = scoring.copy()
    s2.index=[_ for _ in vh]
    s2.columns=[_ for _ in wh]
    s2
    return scoring,s2

##
def LCS(w,v,Fmatrix,debug=False):
    wh = '-'+ w
    vh = '-'+ v
    """
    """
    lcs = ''
    i = Fmatrix.shape[0]-1
    j = Fmatrix.shape[1]-1
    while (i > 0) and (j > 0):
        #print((j,i), s[j][i],LCS[::-1],'<===LCS\t',vh[i],wh[j])
        if vh[i] == wh[j]:
                lcs += wh[j]
                i -= 1
                j -= 1
                if debug: print('\tDiag')
        else:
            if i >= 0: up = Fmatrix[j][i-1]
            else: up = -np.inf
            if i >= 0: left = Fmatrix[j-1][i]
            else: left = -np.inf
            if left > up: 
                if debug: print("\t",(up,left),'left')
                j -= 1
            else:
                if debug: print("\t",(up,left),'up')
                i -= 1
    return lcs[::-1]


In [5]:
import pandas as pd
gap = 0#-4
mismatch = 0#-3
match = 1

In [91]:
x = 'ATGTTATA'
y = 'ATCGTCC'

m = len(x)
n = len(y)
table = np.zeros((m+1, n+1), dtype=int)
for i in range(1, m+1):
    for j in range(1, n+1):
        if x[i-1] == y[j-1]:
            table[i, j] = table[i-1, j-1] + match
        else:
            table[i, j] = max(table[i, j-1], table[i-1, j])
table

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 2, 2, 2, 2, 2, 2],
       [0, 1, 2, 2, 3, 3, 3, 3],
       [0, 1, 2, 2, 3, 4, 4, 4],
       [0, 1, 2, 2, 3, 4, 4, 4],
       [0, 1, 2, 2, 3, 4, 4, 4],
       [0, 1, 2, 2, 3, 4, 4, 4],
       [0, 1, 2, 2, 3, 4, 4, 4]])

In [94]:
i, j = m, n
w = ''
a=''
b=''
while (i > 0) and (j > 0):
    if x[i-1] == y[j-1]:
        w = x[i-1] + w
        i -= 1
        j -= 1
        a += x[i]
        b += y[j]
    else:
        if table[i, j-1] > table[i-1, j]:
            a += '-'
            b += y[j-1]
            j -= 1
        else:
            a += x[i-1]
            b += '-'
            i -= 1
(i,j,a[::-1],b[::-1],w,table)


(0,
 0,
 'AT-GT--TATA',
 'ATCGTCC----',
 'ATGT',
 array([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 2, 2, 2, 2, 2, 2],
        [0, 1, 2, 2, 3, 3, 3, 3],
        [0, 1, 2, 2, 3, 4, 4, 4],
        [0, 1, 2, 2, 3, 4, 4, 4],
        [0, 1, 2, 2, 3, 4, 4, 4],
        [0, 1, 2, 2, 3, 4, 4, 4],
        [0, 1, 2, 2, 3, 4, 4, 4]]))

In [254]:
## https://www.youtube.com/watch?v=LhpGz5--isw
# x = 'ATGGCCTC'
# y = 'ACGGCTC'
# match = 1
# gap = -4
# mismatch = -3

## Wiki https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm
x='GATTACA'
y='GCATGCU'
match = 1
gap = -1
mismatch = -1


m = len(x)
n = len(y)
Fmatrix = np.zeros((m+1,n+1))

## initialize
for i in range(1,m+1):
    Fmatrix[i,0] += Fmatrix[i-1,0] + gap
for j in range(1,n+1):
    Fmatrix[0,j] += Fmatrix[0,j-1] + gap

##
for i in range(1,m+1):
    for j in range(1,n+1):
        if x[i-1] == y[j-1]:
            Fmatrix[i,j] = Fmatrix[i-1,j-1] + match
        else:
            mm = Fmatrix[i-1,j-1] + mismatch
            down = Fmatrix[i-1,j] + gap
            right = Fmatrix[i,j-1] + gap
            Fmatrix[i,j] = max([mm,down,right])
        #print(x[i-1],y[j-1],[mm,down,right])
            
            
pd.DataFrame(Fmatrix,columns=['-']+[_ for _ in y],index=['-']+[_ for _ in x])

,-,G,C,A,T,G,C,U
-,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0
G,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0
A,-2.0,0.0,0.0,1.0,0.0,-1.0,-2.0,-3.0
T,-3.0,-1.0,-1.0,0.0,2.0,1.0,0.0,-1.0
T,-4.0,-2.0,-2.0,-1.0,1.0,1.0,0.0,-1.0
A,-5.0,-3.0,-3.0,-1.0,0.0,0.0,0.0,-1.0
C,-6.0,-4.0,-2.0,-2.0,-1.0,-1.0,1.0,0.0
A,-7.0,-5.0,-3.0,-1.0,-2.0,-2.0,0.0,0.0


In [255]:































pd.DataFrame(Fmatrix,columns=['-']+[_ for _ in y],index=['-']+[_ for _ in x])

,-,G,C,A,T,G,C,U
-,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0
G,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0
A,-2.0,0.0,0.0,1.0,0.0,-1.0,-2.0,-3.0
T,-3.0,-1.0,-1.0,0.0,2.0,1.0,0.0,-1.0
T,-4.0,-2.0,-2.0,-1.0,1.0,1.0,0.0,-1.0
A,-5.0,-3.0,-3.0,-1.0,0.0,0.0,0.0,-1.0
C,-6.0,-4.0,-2.0,-2.0,-1.0,-1.0,1.0,0.0
A,-7.0,-5.0,-3.0,-1.0,-2.0,-2.0,0.0,0.0


In [304]:
i = Fmatrix.shape[0] - 1
j = Fmatrix.shape[1] - 1

LCS = ""
a = ''
b = ''
paths = []
while (j>0 or i>0):
    print(x[i-1],y[j-1],i,j)
    if x[i-1] == y[j-1]:
        LCS += x[i-1]
        j -=1
        i -=1
        a +=x[i]
        b +=y[j]
        paths.append([(np.inf),(x[i],y[j]) ])
    else:
        diag = Fmatrix[i-1,j-1] #+ mismatch
        left = Fmatrix[i-1,j] #+ gap
        up = Fmatrix[i,j-1] #+ gap
        maxCell = np.argmax([diag,left,up])
        print(maxCell,'diff',[diag,left,up])
        if maxCell == 0:
            j -=1
            i -=1
            a +=x[i]
            b +=y[j]
        else:
            if maxCell == 1:
                i -= 1
                a += x[i]
                b += '-'
            else:
                j -= 1
                a += '-'
                b += y[j]
        paths.append( [(diag,left,up),
                       (x[i],x[i],"-"),
                       (y[j],"-",y[j]) ])
print(LCS[::-1]) #AACTGG
print('--')
print(b[::-1])
print(a[::-1])

A U 7 7
0 diff [1.0, 0.0, 0.0]
C C 6 6
A G 5 5
0 diff [1.0, 1.0, 0.0]
T T 4 4
T A 3 3
1 diff [0.0, 1.0, -1.0]
A A 2 3
G C 1 2
2 diff [-1.0, -2.0, 1.0]
G G 1 1
GATC
--
GCA-TGCU
G-ATTACA


In [300]:
## 
for _ in paths:
    if _[0] == np.inf:
        print(_[1][0],_[1][1])
    else:
        ixMax = [i for i,val in enumerate(_[0]) if val == max(_[0])]
        print([(_[1][mx],_[2][mx]) for mx in ixMax])

[('A', 'U')]
C C
[('A', 'G'), ('A', '-')]
T T
[('T', '-')]
A A
[('-', 'C')]
G G


In [292]:
GCA-TGCU
G-ATTACA


GCA-T-CU
G-ATTACA

[(1.0, 1.0, 0.0), ('A', 'A', '-'), ('G', '-', 'G')]

In [301]:
paths

[[(1.0, 0.0, 0.0), ('A', 'A', '-'), ('U', '-', 'U')],
 [inf, ('C', 'C')],
 [(1.0, 1.0, 0.0), ('A', 'A', '-'), ('G', '-', 'G')],
 [inf, ('T', 'T')],
 [(0.0, 1.0, -1.0), ('T', 'T', '-'), ('T', '-', 'T')],
 [inf, ('A', 'A')],
 [(-1.0, -2.0, 1.0), ('A', 'A', '-'), ('C', '-', 'C')],
 [inf, ('G', 'G')]]

In [436]:
### Find parents of max for backtracking
m = Fmatrix.shape[0] - 1
n = Fmatrix.shape[1] - 1

i = m
j = n

def getParents(coord):
    i=coord[0]
    j=coord[1]
    if i == 0 and j == 0:
        return None
    else:
        diag=Fmatrix[i-1,j-1]
        up=Fmatrix[i-1,j]
        left=Fmatrix[i,j-1]
        Neighbors=[diag,up,left]
        #print(Neighbors)
        maxVal=max(Neighbors)
        #print(maxVal)
        parentCoords = [(i-1,j-1),(i-1,j),(i,j-1)]
        #return {(i,j): [parentCoords[i] for i,val in enumerate(Neighbors) if val == maxVal]}
        out = [parentCoords[i] for i,val in enumerate(Neighbors) if val == maxVal]
        node = (i,j)
        return out

In [410]:
coord, parents = getParents((7,7))
parents

[1.0, 0.0, 0.0]
1.0


[(6, 6)]

In [437]:
p7 = getParents((7,7))
{_:getParents(_) for _ in  p7}

{(6, 6): [(5, 5), (5, 6)]}

In [452]:


while (i,j) != (0,0):
    tmp = getParents((i,j))
    print("--\n",(i,j),tmp)
    print({ (i,j): {_:getParents(_) for _ in tmp}})
    i -= 1
    j -= 1
    

--
 (7, 7) [(6, 6)]
{(7, 7): {(6, 6): [(5, 5), (5, 6)]}}
--
 (6, 6) [(5, 5), (5, 6)]
{(6, 6): {(5, 5): [(4, 4), (4, 5)], (5, 6): [(4, 5)]}}
--
 (5, 5) [(4, 4), (4, 5)]
{(5, 5): {(4, 4): [(3, 4)], (4, 5): [(3, 4)]}}
--
 (4, 4) [(3, 4)]
{(4, 4): {(3, 4): [(2, 3)]}}
--
 (3, 3) [(2, 3)]
{(3, 3): {(2, 3): [(1, 2), (2, 2)]}}
--
 (2, 2) [(1, 1)]
{(2, 2): {(1, 1): [(0, 0)]}}
--
 (1, 1) [(0, 0)]
{(1, 1): {(0, 0): None}}


In [443]:
pd.DataFrame(Fmatrix,columns=[i for i,_ in enumerate("-"+y)],
                     index=[i for i,_ in enumerate("-"+x)])

,0,1,2,3,4,5,6,7
0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0
1,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0
2,-2.0,0.0,0.0,1.0,0.0,-1.0,-2.0,-3.0
3,-3.0,-1.0,-1.0,0.0,2.0,1.0,0.0,-1.0
4,-4.0,-2.0,-2.0,-1.0,1.0,1.0,0.0,-1.0
5,-5.0,-3.0,-3.0,-1.0,0.0,0.0,0.0,-1.0
6,-6.0,-4.0,-2.0,-2.0,-1.0,-1.0,1.0,0.0
7,-7.0,-5.0,-3.0,-1.0,-2.0,-2.0,0.0,0.0


In [481]:
def travelPath(node):
    if getParents(node) is not None:
        return {each:travelPath(node) for each in getParents(node)}
    else:
        return None
         

In [484]:
travelPath((1,1))

RecursionError: maximum recursion depth exceeded in comparison

In [482]:
nodeList = [(7,7)]
for node in nodeList:
    if getParents(node) is not None:
        print ({each:getParents(each) for each in getParents(node)})


{(6, 6): [(5, 5), (5, 6)]}


[(6, 6)]

In [347]:
def path(coord):
    while getParents(coord) is not None:
        for

True

In [332]:
pd.DataFrame(Fmatrix,columns=[i for i,_ in enumerate("-"+y)],
                     index=[i for i,_ in enumerate("-"+x)])

,0,1,2,3,4,5,6,7
0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0
1,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0
2,-2.0,0.0,0.0,1.0,0.0,-1.0,-2.0,-3.0
3,-3.0,-1.0,-1.0,0.0,2.0,1.0,0.0,-1.0
4,-4.0,-2.0,-2.0,-1.0,1.0,1.0,0.0,-1.0
5,-5.0,-3.0,-3.0,-1.0,0.0,0.0,0.0,-1.0
6,-6.0,-4.0,-2.0,-2.0,-1.0,-1.0,1.0,0.0
7,-7.0,-5.0,-3.0,-1.0,-2.0,-2.0,0.0,0.0
